In [26]:
# 資料庫串接
import MySQLdb
import pandas as pd

def connect_db(company_id_input,year_input,season_input):
    conn = MySQLdb.Connect(host = 'localhost',
                        port = 3306,
                        user = 'root',
                        db = 'kpmg',
                        charset='utf8')
    # 輸入你的資料庫連線資訊
    cur = conn.cursor()
    cur.execute(f"SELECT `path` FROM `financial_statements` WHERE `company_id` = {company_id_input} AND `year` = {year_input} AND `season` = {season_input} ")
    result_set = cur.fetchall()
    # result_set
    df = pd.read_csv(f"../../{result_set[0][0]}")
    df2 = pd.read_csv(f"../../{result_set[1][0]}")
    df3 = pd.read_csv(f"../../{result_set[2][0]}")
    df4 = pd.read_csv(f"../../{result_set[3][0]}")
    df_ls = [df,df2,df3,df4]
    return df_ls

def main(company_id_input,year_input,season_input,input_ls):
    store_ls = []
    negative_ls = ["營業費用合計","財務成本淨額","所得稅費用（利益）合計","不重分類至損益之項目","與不重分類之項目相關之所得稅"]
    df_ls = connect_db(company_id_input,year_input,season_input)
    for search in input_ls:
        temp_ls = []
        for each_df in df_ls:
            try:
                filt = (each_df["會計項目Accounting Title"].str.contains(search))
                filt_result = each_df.loc[filt]
                if filt_result.empty:
                    continue
                else:
                    temp_num = filt_result.iloc[0,[3]][0]
                    if pd.isna(temp_num):
                        continue
                    else:
                        if "(" in str(temp_num):
                            temp_num = temp_num.replace("(","")
                            temp_num = temp_num.replace(")","")
                            temp_num = temp_num.replace(",","")
                            temp_num = str(-(int(temp_num)))
                        elif search in negative_ls:
                            temp_num =  str(-(int(temp_num)))
                        temp_ls.append(temp_num)
            except:
                pass
        if temp_ls == []:
            store_ls.append(["Null"])
        else:
            store_ls.append(temp_ls)
    return store_ls
main(company_id_input,year_input,season_input,input_ls)

[['1208477', '-1956'], ['Null'], ['3819177', '-148'], ['Null']]
